In [ ]:
import pandas as pd

In [ ]:
books = pd.read_csv('files/book.csv')
books

In [ ]:
category = books['categoryName'].unique()
count = books['categoryName'].nunique()
print(category, count)

In [ ]:
category_entity = {}
for i, v in enumerate(category):
    category_entity[i] = v

print(category_entity)

In [ ]:
authors = books['authors']

In [ ]:
for i in range(len(books['authors'])):
    if str(books['authors'][i])[-1]=='외':
        authors.iloc[i] = books.loc[i,'authors'].split('외')[0]

In [ ]:
authors = authors[authors.notna()]
authors

In [ ]:
authors = authors[authors.str.contains(' 외 ')==False]

In [ ]:
authors = authors[authors.str.contains('엮은이|옮긴이')==False]

In [ ]:
author = list(authors['authors'])
for i in range(len(authors)):
    a = str(authors.loc[i, 'authors'])
    if ',' in a :
        author.remove(a)
        a = str(a).split(',')
        for k in a:
            author.append(str(k))

In [ ]:
author_set = list(set(author))
print(len(author_set))

In [ ]:
for s in author_set:
    s.strip()

print(author_set)

In [ ]:
print(len(set(author_set)))
author = list(set(author_set))

In [ ]:
len(author)

In [ ]:
author_set = author.copy()
author_dict = {}
for i, v in enumerate(author_set):
    author_dict[i] = v

In [ ]:
author_dict

In [ ]:
modified_dict = {key + 1000: value for key, value in author_dict.items()}
print(len(modified_dict))

In [ ]:
pd.DataFrame.from_dict(modified_dict, orient='index').to_csv('author_entity.csv')

In [ ]:
books_x = pd.read_csv('files/new_data/books.csv')
boook = pd.merge(books, books_x[['id','publishedDate']], on='id', how='left')

In [ ]:
book = boook.copy()
book['publishedCent'] = [None] * len(book)
for i in range(len(book)):
    if str(book.loc[i, 'publishedDate_y'])[0:2] == '19':
        book.loc[i, 'publishedCent'] = '20C'
    elif str(book.loc[i, 'publishedDate_y'])[0:2] == '20':
        book.loc[i, 'publishedCent'] = '21C'
    else:
        pass

===================================================

In [ ]:
merged_book = pd.read_csv('example.csv', encoding='euc-kr')
merged_book.head()

In [ ]:
from tqdm import tqdm
aaa = {str(v):k for k, v in author_dict.items()}
merged_book['final_author_entity'] = [''] * len(merged_book)

In [ ]:
for i in tqdm(range(len(merged_book))):
    author_list = str(merged_book.loc[i, 'authors']).split(',')
    for aa in author_list:
        # 딕셔너리의 value 중에서 탐색 -> 키 반환
        try:
            key = aaa.get(aa.strip())
            if key is not None:
                merged_book.loc[i, 'final_author_entity'] += (str(int(key)+1000)+', ')
            else:
                pass
        except Exception as e:
            print(e)
            pass
    merged_book.loc[i, 'final_author_entity'].rstrip(', ')

display(merged_book)

In [ ]:
merged_book[['authors', 'final_author_entity']]

In [ ]:
merged_book.columns

In [ ]:
final_merged_book = merged_book.drop(columns=['Unnamed: 0', 'Unnamed: 0_x','Unnamed: 0.1', 'Column1',
       'Unnamed: 0_y','author_entity_x'])
final_merged_book

In [ ]:
category_entity2 = {str(v):k for k, v in category_entity.items()}
book['category_entity'] = [''] * len(book)

In [ ]:
for i in tqdm(range(len(book))):
    categoryName = str(book.loc[i, 'categoryName'])
    #entity_list = ''
    try:
        key = category_entity2.get(categoryName)
        book.loc[i, 'category_entity'] += (str(int(key)))
    except Exception as e:
        print(e)
        pass

display(book)

In [ ]:
book2 = pd.read_csv('files/book.csv')
max(book2['readerCount'])

In [ ]:
book2[book2.duplicated(subset=['title','authors'],keep=False)==True].itemId.isna().value_counts()

In [ ]:
# 푸아송 분포로 readercount 확인
from scipy.stats import poisson
import numpy as np
import matplotlib.pyplot as plt

mu = 5
readercount = book2['readerCount']
pmf = poisson.pmf(readercount,mu)

plt.bar(readercount, pmf)
plt.show()

In [ ]:
book2.to_csv('book_entity_1.csv', encoding='cp949')

작가명이 한글자인 경우는 ENTITY 제외해야 함..... ex - 김, 가, 어쩌고 등

## PickCount 분포 확인

In [ ]:
from scipy.stats import poisson
import numpy as np
import matplotlib.pyplot as plt

mu = 5
readercount = book['readerCount']
pmf = poisson.pmf(readercount,mu)

plt.bar(readercount, pmf)
plt.xlim(0,20)
plt.show()

0 : 0 ~ 3 적은 인기
1 : 3 ~ 7 중간 인기
2 : 7 ~ 10 적당한 인기
3 : 많은 인기

In [ ]:
book['readerCount_entity'] = ['None'] * len(book)
book['readerCount_entity']

In [ ]:
for i in range(len(book)):
    if not book.loc[i, 'readerCount'] or np.isnan(book.loc[i, 'readerCount']):
        continue
    print(book.loc[i, 'readerCount'])
    if 0 <= int(book.loc[i, 'readerCount']) <= 3:
        book.loc[i, 'readerCount_entity'] = 0
    elif 3 < int(book.loc[i, 'readerCount']) <=7:
        book.loc[i, 'readerCount_entity'] = 1
    elif 7 < int(book.loc[i, 'readerCount']) <=10:
        book.loc[i, 'readerCount_entity'] = 2
    else:
        book.loc[i, 'readerCount_entity'] = 3

book

In [ ]:
import pandas as pd
book = pd.read_csv('book_entity.csv',encoding='euc-kr', encoding_errors='ignore')

In [ ]:
for i, row in book.iterrows():
    try:
        new = 50 + int(row['readerCount_entity'])
    except:
        new = ''
    book.at[i, 'readerCount_entity'] = new
book

In [ ]:
book.to_csv('final_book_entity.csv', encoding='euc-kr')

In [ ]:
book[book['readerCount_entity']==53]

In [ ]:
for i, row in df.iterrows():
    df.loc[i, 'category_entity'] = row['category_entity'] + 500
df

In [ ]:
for i, row in df.iterrows():
    if row['publishedCent']=='20C':
        df.loc[i, 'publishedCent'] = 100
    if row['publishedCent']=='21C':
        df.loc[i, 'publishedCent'] = 101

In [ ]:
for i, row in fb_df.iterrows():
    if row['publishedCent']=='20C':
        fb_df.loc[i, 'publishedCent'] = 100
    if row['publishedCent']=='21C':
        fb_df.loc[i, 'publishedCent'] = 101

In [ ]:
for i, row in fb_df.iterrows():
    fb_df.loc[i, 'category_entity'] = row['category_entity'] + 500

In [ ]:
for i, row in fb_df.iterrows():
    if fb_df.loc[i, 'readerCount'] ==1:
        fb_df.loc[i, 'readerCount_entity'] = 50
    elif 1 < fb_df.loc[i, 'readerCount'] <=15:
        fb_df.loc[i, 'readerCount_entity'] = 51
    elif fb_df.loc[i, 'readerCount'] > 16:
        fb_df.loc[i, 'readerCount_entity'] = 52

In [ ]:
fb_df.to_csv('final_book_entity.csv')

## 16자리 book id => int 매핑

In [ ]:
for i in range(len(fb_df[['id']])):
    item_list = pd.concat([item_list, pd.DataFrame([{"id" : fb_df.loc[i, 'id'], "remap_id":i}])])
item_list

In [ ]:
item_list.to_csv('item_list.csv')